In [ ]:
import numpy as np
from plotly import graph_objs as go

import dsplib
import plotlib

In [ ]:
import importlib
importlib.reload(dsplib)
importlib.reload(plotlib)

In [ ]:
fs_hz = 48e3
signal_duration_sec = 0.2
smp_num = dsplib.calc_smp_num(signal_duration_sec, fs_hz)

In [ ]:
S = dsplib.generate_delayed_sin_matrix(smp_num=smp_num, tone_freq_n=9100/fs_hz, mag=0.5, history_smp_num=3, noise_level=1e-4)
sig = S[0,:]

In [ ]:
@dsplib.resample(factor=32)
def tanh_resample(x):
    return np.tanh(x)

In [ ]:
mag = 3
t = dsplib.generate_time_signal(signal_duration_sec=signal_duration_sec, fs_hz=fs_hz)

fig = go.Figure()
freq_points_hz = np.arange(300, fs_hz/2, 200)

labels = []
for tone_freq_hz in freq_points_hz:
    sig = mag * np.sin(2*np.pi * tone_freq_hz * t)
    f, S_in = dsplib.calc_spectrum(sig, fs=fs_hz)
    _, S_out = dsplib.calc_spectrum(np.tanh(sig), fs=fs_hz)

    tanh_ups_dwn = tanh_resample(sig)
    _, S_out_ups = dsplib.calc_spectrum(tanh_ups_dwn, fs=fs_hz)

    fig.add_trace(go.Scatter(x=f, y=S_in, visible=False, name="input"))
    fig.add_trace(go.Scatter(x=f, y=S_out, visible=False, line=dict(width=2, dash='dash'), name="tanh"))
    fig.add_trace(go.Scatter(x=f, y=S_out_ups, visible=False, line=dict(width=2, dash='dot'), name="tanh ups"))

    labels.append(int(tone_freq_hz))

fig = plotlib.apply_sliders(fig, lines_num=3, labels=labels, prefix="Frequency: ", suffix=" Hz")

fig.update_layout(
    title=f"Signal spectrum before and after tanh",
    xaxis_title='Frequency, Hz',
    yaxis_title='Spectral density'
)

fig.show()